In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [22]:
raw_df = pd.read_csv('C://Workspace/02.DataAnalysis/05.Crawling/data/제주데이터.csv')
raw_df.head()

,Unnamed: 0,content,date,like,place,tags
0,0,더 늦기전에 제주도로 떠나쟈😍.#제주핫플레이스 간단히 1탄 모아봄!#제주여행 요 필...,2018-12-07,"1,402",NaN,"['#제주핫플레이스', '#제주여행', '#제주여행', '#제주도여행', '#제주가..."
1,1,12월에 제주도를 가야하는이유🍊.#제주핫플 모음 끝판왕😍이거 하나면 겨울 #제주여행...,2018-12-03,368,NaN,"['#제주핫플', '#제주여행', '#제주', '#제주도', '#제주도맛집', '#..."
2,2,11월 놓치지 말아야 할 제주 관광.가을바람이 최고조에 이르는 11월추운 겨울을 앞...,2018-11-02,166,NaN,"['#honestin', '#어니스틴', '#제주여행', '#제주', '#제주도',..."
3,3,국민학교세대#제주관광#제주살이#제주이주민#아라동주민#삼남매집 #새해첫날#드라이브#명...,2019-01-01,28,명월국민학교,"['#제주관광', '#제주살이', '#제주이주민', '#아라동주민', '#삼남매집'..."
4,4,#제주관광 #제주 #돔나이트 #스트레스 #풀자 #춤추며 #땀날려 #가끔은괜찮아 #인...,2019-01-01,12,NaN,"['#제주관광', '#제주', '#돔나이트', '#스트레스', '#풀자', '#춤추..."


In [23]:
location_counts = raw_df.place.value_counts().to_frame()
print(location_counts.shape)
location_counts.head()

(1027, 1)


,place
Jeju,241
Jeju-do,172
Jungle Book by Alice,101
Seogwipo,65
제주도 크리스마스 박물관,59


In [24]:
location_counts.tail()

,place
흑돼지가있는풍경,1
물꼬기,1
블랙하우스 떡볶이레스토랑,1
김녕 성세기 해변,1
디스이즈핫,1


In [25]:
location_counts.to_csv('location_counts.csv')

In [26]:
loc_df = pd.read_csv('location_counts.csv')
loc_df.columns = ['지명','빈도']
loc_df.head()

,지명,빈도
0,Jeju,241
1,Jeju-do,172
2,Jungle Book by Alice,101
3,Seogwipo,65
4,제주도 크리스마스 박물관,59


### 카카오검색 API를 이용한 키워드 장소 검색

In [11]:
with open('kakaoapikey.txt') as f:
    api_key = f.read()

In [12]:
import requests
from urllib.parse import quote

In [13]:
keyword = '합정 스타벅스'
search_url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
url = f'{search_url}?query={quote(keyword)}'
result = requests.get(url, headers={"Authorization": f'KakaoAK {api_key}'}).json()
result['documents'][0]

{'address_name': '서울 마포구 서교동 395-166',
 'category_group_code': 'CE7',
 'category_group_name': '카페',
 'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
 'distance': '',
 'id': '26572121',
 'phone': '1522-3232',
 'place_name': '스타벅스 서교점',
 'place_url': 'http://place.map.kakao.com/26572121',
 'road_address_name': '서울 마포구 양화로 78',
 'x': '126.916980454434',
 'y': '37.5514601750423'}

In [29]:
import re

In [ ]:
re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]','','')

In [27]:
def find_place(keyword):
    url = f'{search_url}?query={quote(keyword)}'
    result = requests.get(url, headers={"Authorization": f'KakaoAK {api_key}'}).json()
    try:
        name = result['documents'][0]['place_name']
        lng = float(result['documents'][0]['x'])
        lat = float(result['documents'][0]['y'])
    except:
        name = ''
        lng = np.nan
        lat = np.nan
    
    return name, lng, lat, keyword


In [17]:
find_place('제주공항')

('제주국제공항', 126.492769004244, 33.5070789578184, '제주공항')

In [30]:
# loc_df에 공식명칭, 위도, 경도 정보 추가
name_list, lng_list, lat_list = [], [], []
for place in loc_df.지명:
    place = re.sub('[^가-힣0-9]','',place).strip()
    if place:
        name, lng, lat, _ = find_place(place)
    else:
        name, lng, lat = '', np.nan, np.nan
    name_list.append(name)
    lng_list.append(lng)
    lat_list.append(lat)

In [31]:
loc_df['공식명칭'] = name_list
loc_df['위도'] = lat_list
loc_df['경도'] = lng_list
loc_df.head()

,지명,빈도,공식명칭,위도,경도
0,Jeju,241,,NaN,NaN
1,Jeju-do,172,,NaN,NaN
2,Jungle Book by Alice,101,,NaN,NaN
3,Seogwipo,65,,NaN,NaN
4,제주도 크리스마스 박물관,59,바이나흐튼 크리스마스박물관,33.291572,126.327858


In [32]:
# Null 데이터 삭제
loc_df.isnull().sum()

지명        0
빈도        0
공식명칭      0
위도      197
경도      197
dtype: int64

In [40]:
total_df = loc_df.dropna()
total_df.isnull().sum().sum()

0

In [41]:
total_df.head()

,지명,빈도,공식명칭,위도,경도
4,제주도 크리스마스 박물관,59,바이나흐튼 크리스마스박물관,33.291572,126.327858
5,제주에인감귤밭,48,제주에인감귤밭,33.256561,126.539014
7,성산일출봉 城山日出峰 Seongsan Ilchulbong,36,성산일출봉,33.459135,126.940538
8,폼포코식당_pompokokitchen,30,폼포코식당,33.450408,126.919065
9,1100고지,25,한라산1100고지,33.358078,126.462220


In [42]:
# 공식명칭 중복 여부
total_df.공식명칭.value_counts()[:10]

올레길 7코스(서귀포-월평 올레)    6
동문재래시장                4
함덕해수욕장                4
이호테우해수욕장              3
우도                    3
천지연폭포                 3
세화해변                  3
비자림                   3
협재해수욕장                3
제주중문관광단지              3
Name: 공식명칭, dtype: int64

In [43]:
location_data = total_df.pivot_table(index=['공식명칭','위도','경도'],values='빈도',aggfunc='sum')
location_data.head()

,,,빈도
공식명칭,위도,경도,
1158족욕카페,33.467075,126.336504,1
40계단,35.103911,129.034571,1
702키친,33.498835,126.534134,1
CU 수원광장점,37.268151,127.000311,1
KT 브로 라페스타점,37.662346,126.767821,1


In [44]:
location_data.reset_index(inplace=True)
location_data.공식명칭.value_counts()[:5]

1158족욕카페    1
정관신도시       1
자구리몬들락      1
자매국수 본점     1
자연뜨락        1
Name: 공식명칭, dtype: int64

In [39]:
location_data.to_csv('공식명칭.csv', index=False)

### folium 지도 시각화(제주 지명)

In [45]:
df.sort_values(by='빈도', ascending=False)
df.head()

,지명,빈도,공식명칭,위도,경도
4,제주도 크리스마스 박물관,59,바이나흐튼 크리스마스박물관,33.291572,126.327858
5,제주에인감귤밭,48,제주에인감귤밭,33.256561,126.539014
7,성산일출봉 城山日出峰 Seongsan Ilchulbong,36,성산일출봉,33.459135,126.940538
8,폼포코식당_pompokokitchen,30,폼포코식당,33.450408,126.919065
9,1100고지,25,한라산1100고지,33.358078,126.462220


In [ ]:
df[['빈도']].boxplot()

In [55]:
location_data.head()

,공식명칭,위도,경도,빈도
0,1158족욕카페,33.467075,126.336504,1
1,40계단,35.103911,129.034571,1
2,702키친,33.498835,126.534134,1
3,CU 수원광장점,37.268151,127.000311,1
4,KT 브로 라페스타점,37.662346,126.767821,1


In [47]:
import folium

In [70]:
float(location_data['경도'][0])

126.336504121082

In [ ]:
Mt_Hanla = [33.362500,126.533694]
map_jeju = folium.Map(location = Mt_Hanla, zoom_start=11)

name = location_data['공식명칭'][0]
count = location_data['빈도'][0]
size = int(count)*2
location = (float(location_data['위도'][0]),float(location_data['경도'][0]))
lat = float(location_data['위도'][0])
folium.CircleMarker((33.467075,126.336504), radius=size, color='red',popup=name).add_to(map_jeju)
map_jeju

In [ ]:
for i in location_data.index:
    print(i)

In [72]:
Mt_Hanla = [33.362500,126.533694]
map_jeju = folium.Map(location = Mt_Hanla, zoom_start=11)

for i in location_data.index:
    name = location_data['공식명칭'][i]
    count = location_data['빈도'][i]
    size = int(count)*2
    location = (location_data['위도'][i],location_data['경도'][i])
    folium.CircleMarker(location, radius=size, color='red',popup=name).add_to(map_jeju)

map_jeju